In [1]:
import pandas as pd
from tensorflow.python.ops.metrics_impl import root_mean_squared_error

# 读取数据
data = pd.read_csv("data/1_dataset_fillna.csv")
data = data.drop(["M", "Anion", "CC"], axis=1)
data

,O,N,B,S,P,SSA,PV,RMIC,Dap,ID/IG,AML,PW,CD,Cs
0,7.15,3.52,0.00,0.0,0.0,1546.1,0.5753,0.939857,1.488390,1.029,1.00,1.6,0.10,85.0
1,8.12,3.79,3.66,0.0,0.0,1315.2,0.5116,0.931392,1.555961,1.038,1.00,1.6,0.10,114.0
2,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.10,90.2
3,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.25,86.3
4,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.50,79.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,10.00,67.1
668,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,20.00,63.1
669,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,30.00,59.6
670,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,40.00,59.3


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 按目标值分布按10个等级进行数据分割
data['target_class'] = pd.qcut(data['Cs'], q=10, labels=False)
X = data.drop('Cs', axis=1)
y = data['Cs']
stratify_column = data['target_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

# 数据标准化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 将数据转换成DF以便后续使用
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [10]:
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error

# 初始化xgboost
xgb = XGBRegressor(
    objective='reg:squarederror',  # 回归任务的损失函数
    n_estimators=100,              # 树的数量（boosting轮数）
    learning_rate=0.1,             # 学习率
    max_depth=4,                   # 树的最大深度
    random_state=21                # 随机种子
)

# 训练模型
xgb.fit(X_train_scaled, y_train)

# 在测试集上进行预测
y_pred = xgb.predict(X_test_scaled)

# 计算均方误差（MSE）
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {rmse}")

Mean Squared Error: 8.035917910297387
